In [11]:
import pandas as pd
import csv
import nltk
import string
import math
from tqdm import tqdm
from nltk.corpus import stopwords
from collections import defaultdict
import requests
from bs4 import BeautifulSoup
from random import randint
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import *
nltk.download('stopwords')
from sklearn.cluster import KMeans
import numpy as np

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Part 1 ##

#### 1.1 ####

In [ ]:
anime = []

for page in tqdm(range(0, 400)):
    url = 'https://myanimelist.net/topanime.php?limit=' + str(page * 50)
    response = requests.get(url)
    
    soup = BeautifulSoup(response.text, 'html.parser')
    for tag in soup.find_all('tr'):
        links = tag.find_all('a')
        for link in links:        
            if type(link.get('id')) == str and len(link.contents[0]) > 1:
                anime.append((link.contents[0], link.get('href')) )

In [ ]:
f = open("links.txt", "w")
for el in anime:
    f.write(str(el[1])+'\n')
f.close()
#We get 19083 rows, which implies we have less than 20000 animes

#### 1.2 ####

In [ ]:
#first open file from which we get links of the pages to download
f = open("links.txt","r")
lines = f.readlines()
f.close()

for page in tqdm(range(364,382)): #range is for pages to download [each page contains 50 articles]
    #Manage the directories
    os.system("mkdir Page"+str(page+1)) #the (page+1) is to make so we don't have the first element 0 but 1
    os.chdir("Page"+str(page+1))
    
    #this is needed for the indexing of the articles
    start=50*page
    
    for i in range(start,start+50):
        url=lines[i]
        check = requests.get(url)
        
        #Check if the site is blocking the download, start again when we are allowed to
        if check.status_code != 200:
            print("Last well executed page is {}".format(page-1))
            while(check.status_code != 200): #Stop while we can't download
                check = requests.get(url)    #update the request to connect to the page until
                                             #we manage to connect
                    
                #next line was just a try of changing the ip when the site would stop the downloads
                #in short didn't manage to implement it
                #os.system(("sudo ifconfig lo 127.0.0.{}  netmask 255.0.0.0").format(randint(0,50))) 
       
        #Save the content of the page 
        art=open("article"+str(i+1)+'.html',"wb")
        art.write(check.content)
        art.close()
    
    os.chdir("..")

#### 1.3 #####

In [ ]:
#function that parses information from the single article 
def Parse2(tsv, soup):
    animeTitle,animeType,(animeNumEpisode),(releaseDate),(endDate),(animeNumMembers),(animeScore),(animeUsers),(animeRank),(animePopularity),animeDescription,(animeRelated),(animeCharacters),(animeVoices),(animeStaff)="","","","","","","","","","","","","","",""

    left_col=soup.find('div', {'style':"width: 225px"})
    l_div=left_col.find_all("div",{"class":"spaceit_pad"}) #we will iterate over this, more later
    
    start=""
    end=""
    
    #This is a for over the "div tag" on the left column of the page, in which are listed most of the info we care about
    #going through the html like this speeds up the process of reading the single element
    for i in range(len(l_div)):
        
        if "Type" in l_div[i].getText(): #get type
            animeType = l_div[i].getText().replace("Type:","").replace("\n","").strip()
            #print(animeType)
            
        if "Episodes" in l_div[i].getText(): #get #episodes
            animeNumEpisode = (l_div[i].getText().replace("Episodes:","").replace("\n","").strip())
            if animeNumEpisode=="Unknown":
                animeNumEpisode=""
            #print(animeNumEpisode)
        
        #this piece of code is probably not that readable but I had to adjust it in different times in order
        #to get as much info as possible since there were many different cases
        if "Aired" in l_div[i].getText(): #get end and starting date (when it's there)
            if len(l_div[i].getText().replace("Aired:","").replace("to","").strip().split())==6:
                start=" ".join(l_div[i].getText().replace("Aired:","").replace(",","").replace("to","").strip().split()[0:3])
                end=" ".join(l_div[i].getText().replace("Aired:","").replace(",","").replace("to","").strip().split()[3:6])
                start=datetime.strptime(start,"%b %d %Y")
                end=datetime.strptime(end,"%b %d %Y")
            if len(l_div[i].getText().replace("Aired:","").replace(",","").replace("to","").strip().split())==3 and ("?" not in l_div[i].getText().replace("Aired:","").replace("to","").strip().split()):
                if len(l_div[i].getText().replace("Aired:","").replace(",","").replace("to","").strip().split()[1])!=len(l_div[i].getText().replace("Aired:","").replace(",","").replace("to","").strip().split()[2]):
                    
                    start=" ".join(l_div[i].getText().replace("Aired:","").replace(",","").replace("to","").strip().split()[0:3])
                    start=datetime.strptime(start,"%b %d %Y")
                else:
                    start=""
            else:
                start=end=""
            
            releaseDate=start
            endDate=end
            #print(releaseDate)
            #print(endDate)
            
        #the i!=0 and "ookiku" exceptions are ad-hoc exceptions made for some animes
        #which contained info parsed in a different way from the rest of the group
        if ("Score" in l_div[i].getText()) and (i!=0) and ("Ookiku" not in l_div[i].getText()): #get score
            if l_div[i].getText().replace("Score:","").replace(",","").split()[3]!="-":
                animeUsers = int(l_div[i].getText().replace("Score:","").replace(",","").split()[3])
                animeScore = float(l_div[i].getText().replace("Score:","").replace(",","").split()[0])
            #print(animeUsers)
            #print(animeScore)
        #if "Ranked" in l_div[i].getText():#19 >>>> conviene prenderlo da sopra il rank
            
        if "Popularity" in l_div[i].getText(): #get popularity
            animePopularity = int(l_div[i].getText().replace("Popularity:","").strip().replace("#",""))
            #print(animePopularity)
    
        if "Members" in l_div[i].getText(): #get #members
            animeNumMembers = int(l_div[i].getText().replace("Members:","").strip().replace(",",""))
            #print(animeNumMembers)
   
    
    description=soup.find("p",{"itemprop":"description"}) #get synopsis
    animeDescription=description.getText().replace("\n","")
    #print(animeDescription)
    
    #THESE ARE THE SUGGESTED ONES (WE WANT THE RELATED)
    """rel=[]
    related=soup.find_all("li",{"class":"btn-anime","style":"width:90px"})
    for i in range(len(related)):
        rel.append(related[i]["title"])
    animeRelated = rel
    print(animeRelated)"""
     
    rel=[] #get related animes
    related=soup.find("table",{"class":"anime_detail_related_anime"})#.find_all("td",{"class":"borderClass"})
    if (related)!=None:
        #print(type(related),related)
        related=related.find_all("td",{"class":"borderClass"})
        for i in range(len(related)):
            if i%2!=0:
                rel.append(related[i].getText())
    animeRelated=rel
    
    
    chars=[]
    voices=[]
    staff=[]
    l=[]
    
    #As before, loop over the 2 tables in the html to speed up everything
    seed=soup.find_all("div",{"class":"detail-characters-list clearfix"})
    for i in range(len(seed)):
        if i==0:
            who=soup.find("div",{"class":"detail-characters-list clearfix"}).find_all("img")
            for i in range(len(who)): #get anime characters and voices
                if i%2==0:
                    chars.append(",".join(who[i]["alt"].replace(" ","").split(",")))
                else:
                    voices.append(",".join(who[i]["alt"].replace(" ","").split(",")))
    
        if i==1: #get anime staff
            if (len(soup.find_all("div",{"class":"detail-characters-list clearfix"}))>1):
                for i in range(len(soup.find_all("div",{"class":"detail-characters-list clearfix"})[1].find_all("img"))):
                    staff.append(soup.find_all("div",{"class":"detail-characters-list clearfix"})[1].find_all("img")[i]["alt"])
                    staff.append((soup.find_all("div",{"class":"detail-characters-list clearfix"})[1].find_all("div", {"class":"spaceit_pad"}))[i].getText().strip())
                    
                for i in range(0,2*len(soup.find_all("div",{"class":"detail-characters-list clearfix"})[1].find_all("img")),2):
                    l.append(staff[i:i+2])
            
            
    animeCharacters = chars
    animeVoices = voices
    animeStaff = l 
    #print(animeCharacters)
    #print(animeVoices)
    #print(animeStaff)
    
    
    animeTitle = soup.find("h1",{"class":"title-name h1_bold_none"}).getText() #get title
    animeRank=soup.find("span",{"class":"numbers ranked"}).getText().replace("Ranked #","") #get rank
    #Here we exclude the missing information case
    if "N/A" not in animeRank:
        animeRank = int(animeRank)
    else:
        animeRank=""
    
    #write title and info in the TSV
    tsv.write("animeTitle"+"\t"+"animeType"+"\t"+"animeNumEpisode"+"\t"+"releaseDate"+"\t"+"endDate"+"\t"+"animeNumMembers"+"\t"+"animeScore"+"\t"+"animeUsers"+"\t"+"animeRank"+"\t"+"animePopularity"+"\t"+"animeDescription"+"\t"+"animeRelated"+"\t"+"animeCharacters"+"\t"+"animeVoices"+"\t"+"animeStaff"+"\n")
    tsv.write(animeTitle+"\t"+animeType+"\t"+str(animeNumEpisode)+"\t"+str(releaseDate)+"\t"+str(endDate)+"\t"+str(animeNumMembers)+"\t"+str(animeScore)+"\t"+str(animeUsers)+"\t"+str(animeRank)+"\t"+str(animePopularity)+"\t"+animeDescription+"\t"+str(animeRelated)+"\t"+str(animeCharacters)+"\t"+str(animeVoices)+"\t"+str(animeStaff))

In [ ]:
for page in tqdm(range(0,382)):
    
    start=50*page
    
    for i in range(start,start+50):
        #Open the articles of the page and save all the TSVs in a single directory
        os.chdir("Page"+str(page+1))
        art=open("article"+str(i+1)+'.html',"r")
        os.chdir("..")
        os.chdir("TSV")
        tsv=open("anime_"+str(i+1)+".tsv","w")
        
        soup=BeautifulSoup(art.read(), "html.parser") 
        
        #Just use the parsing function we developed earlier
        Parse2(tsv,soup)
        
        art.close()
        tsv.close()
        os.chdir("..")

Creating a Dataframe to later insert every result into

In [3]:
df = pd.DataFrame()
df = pd.DataFrame(columns = ['animeTitle', 'animeType', 'animeNumEpisode', 'releaseDate', 'endDate', 'animeNumMembers', 'animeScore', \
           'animeUsers', 'animeRank', 'animePopularity', 'animeDescription', 'animeRelated', 'animeCharacters', \
           'animeVoices', 'animeStaff'])

Creating the function to fill the CSV file from the TSV files

In [4]:
path = "C:/Users/paolo/OneDrive/Desktop/TSV/anime_"
lista = range(12687,12701)
def fillcsv(m,n):
  for elem in tqdm(range(m,n)):
    if elem not in lista:
        tsv_file = open(path+str(elem)+".tsv",encoding="utf8")
        read_tsv = csv.reader(tsv_file, delimiter="\t")
        cont = 0
        for anime in read_tsv:
        # using cont to avoid using the first line 
          if cont == 1:
            anime = anime[0:15]
            df.loc[elem] = anime
          cont +=1
fillcsv(1,19083)

100%|██████████| 19082/19082 [02:45<00:00, 114.97it/s]


## PART 2 ##

#CREATING INDEX AND SEARCH ENGINE

Here i removed the stopwords while creating the index for every word in the dataframe anime descriptions

In [137]:
word_index = defaultdict(list)
#n will run through the length of the dataframe
n = df.shape[0]
for i in tqdm(range(0,n)):
  #fetching the anime descriptions
  desc = df.iloc[i]['animeDescription']
  splitted = desc.translate(str.maketrans('', '', string.punctuation)).split(" ")
  #creating the dictionary for the indices
  for parola in splitted:
    parola = parola.lower()
    if parola not in stopwords.words('english'):
      if parola not in word_index:
        word_index[parola] = [[i]]
      else:
        if [i] not in word_index[parola]:
          word_index[parola].append([i])

100%|██████████| 19067/19067 [03:33<00:00, 89.21it/s] 


Here i created 2 functions, one to fetch the URL of an anime and another one to unpack a list of lists as a list

In [138]:
def ReturnUrl(index):
    f = open("C:/Users/paolo/OneDrive/Desktop/TSV/links.txt",encoding="utf8")
    stringa = ""
    for i, line in enumerate(f):
        if i == index:
            stringa = line
    return stringa

def listoflists(list):
  flat_list = []
  for elem in list:
    flat_2 = []
    for single in elem:
      flat_2.append(single[0])
    flat_list.append(flat_2)
  return flat_list

Here i created another 2 functions, one is the search engine and the other one is used to display the results in a dataframe

In [139]:
def search_engine(query,dataset):
  query = query.split(" ")
  query_indices = []
  for index in query:
    query_indices.append(word_index[index])
  # used the set.intersection to see the items that are in both lists
  query_indices = set.intersection(*map(set,listoflists(query_indices)))
  query_index = set(query_indices)
  res = list(query_index)
  return(res)

def displayresults(lista):
    disp_res = pd.DataFrame()
    disp_res = pd.DataFrame(columns = ['Anime Title','Anime Description','Anime URL'])
    res = []
    for i in lista:
        roba = []
        roba.append(df.iloc[i]['animeTitle'])
        roba.append(df.iloc[i]['animeDescription'])
        roba.append(ReturnUrl(i))
        res.append(roba)
    index = 0
    for elem in res:
        disp_res.loc[index] = elem
        index += 1
    return(display(disp_res))
        

Testing the query for a random input from the user

In [140]:
query = str(input("Insert your anime features: "))
displayresults(search_engine(query,df))

Insert your anime features: saiyan race


,Anime Title,Anime Description,Anime URL
0,Dragon Ball Super: Broly,"Forty-one years ago on Planet Vegeta, home of ...",https://myanimelist.net/anime/36946/Dragon_Bal...
1,Dragon Ball Z Special 1: Tatta Hitori no Saish...,"Bardock, Son Goku's father, is a low-ranking S...",https://myanimelist.net/anime/986/Dragon_Ball_...
2,Dragon Ball Z,Five years after winning the World Martial Art...,https://myanimelist.net/anime/813/Dragon_Ball_Z\n


### PART 2.2 CONJUNCTIVE QUERY & RANKING SCORE

Here i created the inverse index dictionary for every word, it will append the term frequency of the word and create the idf for a certain word in the dictionary 

In [5]:
def create_inverse(n,df):
    inverse_index = {}
    for i in tqdm(range(0,n)):
      #fetching the anime descriptions
      desc = df.iloc[i]['animeDescription']
      desc = desc.lower()
      splitted = desc.translate(str.maketrans('', '', string.punctuation)).split(" ")
      #creating the dictionary for the indices
      for parola in splitted:
        parola = parola.lower()
        occurrences = splitted.count(parola)
        if parola not in stopwords.words('english'):
          if parola not in inverse_index:
            # If the word was never encountered we add it and its tf
            inverse_index[parola] = [[i,round(occurrences/len(splitted),4)]]
          else:
            # if the  word was encountered we simply check first that it-s not a duplicate (2 words in the same doc)
            if [i,round((occurrences/len(splitted)),4)] not in inverse_index[parola]:
                # if it was not a duplicate then we add the document and the new tf to the work values
              inverse_index[parola].append([i,round(occurrences/len(splitted),4)])
    # then we just calculate the idf 
    for elem in inverse_index:
        inverse_index[elem].append(round(math.log(df.shape[0]/len(inverse_index[elem])),4))
    return inverse_index

In [6]:
res = create_inverse(df.shape[0],df)

100%|██████████| 19067/19067 [03:39<00:00, 86.97it/s] 


In [7]:
### FUNCTION TO CREATE THE TF_IDF FROM THE WORD_INDEX, just a simple multiplication

In [8]:
def calculate_tfidf(index):
    for word in index:
        for elem in index[word]:
            df = index[word][-1]
            if isinstance(elem, list):
                tfxidf = round(elem[1]*df,3)
                elem[1] = tfxidf
    return index
tf_idf = calculate_tfidf(res)

Then we just calculate the tf_idf for the query terms to calculate the cosine similarity

In [9]:
def termFrequency(term, document):
    normalizeDocument = document.lower().split()
    return normalizeDocument.count(term.lower()) / float(len(normalizeDocument))
def compute_query_tf(query):
    query_norm_tf = {}
    tokens = query.split()
    for word in tokens:
        query_norm_tf[word] = termFrequency(word , query)
    return query_norm_tf
def compute_query_tfidf(query):
    tf_idf_query = []
    for elem in query:
        tf_idf_query.append([elem,query[elem]*res[elem][-1]])
    return tf_idf_query

We define the search engine to fetch the results to calculate the cosine similarity on

In [31]:
def search_engine_2(query,dataset):
  query = query.split(" ")
  query_indices = []
  for term in query:
    if term not in stopwords.words('english'):
        query_indices.append(res[term])
  docs_tfidf = defaultdict(list)
  for match in query_indices:
        for document in match:
            if isinstance(document, list):
                docs_tfidf[document[0]].append(document[1])
  docs_tfidf2 = defaultdict(list)
  for elem in docs_tfidf:
    if len(docs_tfidf[elem])>(len(query_indices)-1):
        docs_tfidf2[elem].append(docs_tfidf[elem])
  return docs_tfidf2

def displayresults(lista):
    disp_res = pd.DataFrame()
    disp_res = pd.DataFrame(columns = ['Anime Title','Anime Description','Anime URL','Score'])
    res = []
    for i in lista:
        roba = []
        roba.append(df.iloc[i[0]]['animeTitle'])
        roba.append(df.iloc[i[0]]['animeDescription'])
        roba.append(ReturnUrl(i[0]))
        roba.append(i[1])
        res.append(roba)
    index = 0
    for elem in res:
        disp_res.loc[index] = elem
        index += 1
    return(display(disp_res))
        

In [26]:
df.iloc[364]["animeTitle"]

'Dragon Ball Z'

Giving an input 

In [33]:
words = str(input("Insert your anime features: "))
index = search_engine_2(words,df)

Insert your anime features: fighting magic


In [34]:
query_idf = compute_query_tfidf(compute_query_tf(words))
final_query = []
for elem in query_idf:
    final_query.append(elem[1])
import numpy as np
def cosine_similarity(query,matches):
    matches_scores = {}
    for elem in matches:
        doc = matches[elem][0]
        matches_scores[elem] = round(np.dot(query,doc)/(np.linalg.norm(query)*np.linalg.norm(doc)),2)
    return(sorted(matches_scores.items(), key=lambda matches_scores: matches_scores[1],reverse = True))
final_result = (cosine_similarity(final_query,index))
displayresults(final_result)

,Anime Title,Anime Description,Anime URL,Score
0,No Game No Life: Zero,"In ancient Disboard, Riku is an angry, young w...",https://myanimelist.net/anime/33674/No_Game_No...,1.00
1,Kekkai Sensen & Beyond,"Three years ago, a gateway between Earth and t...",https://myanimelist.net/anime/34451/Kekkai_Sen...,1.00
2,Kore wa Zombie Desu ka?,Ayumu Aikawa is a 16-year-old high school stud...,https://myanimelist.net/anime/8841/Kore_wa_Zom...,1.00
3,Mahou Sensei Negima!,10-year-old Negi Springfield is a wizard-in-tr...,https://myanimelist.net/anime/157/Mahou_Sensei...,1.00
4,Silent Möbius: The Motion Picture,"In a futuristic Tokyo, several policewomen fig...",https://myanimelist.net/anime/2099/Silent_Möbi...,1.00
5,Aquarian Age: Saga II - Don't Forget Me...,During the chaotic period known in Western ast...,https://myanimelist.net/anime/1268/Aquarian_Ag...,1.00
6,Gakuentoshi Varanoir Joukan,Based on a Playstation 2 game by the same titl...,https://myanimelist.net/anime/3533/Gakuentoshi...,1.00
7,Generation of Chaos III: Toki no Fuuin,Fuujin Year 130 (Medieval times when magic and...,https://myanimelist.net/anime/2778/Generation_...,1.00
8,Zhu Zhu Xia: Yong Chuang Juren Dao,Jack the Robber loses his magic bean. G.G. Bon...,https://myanimelist.net/anime/38117/Zhu_Zhu_Xi...,1.00
9,Mahou no Tenshi Creamy Mami,"Creamy Mami is about a young girl, Yuu, who af...",https://myanimelist.net/anime/2044/Mahou_no_Te...,0.95


## PART 3 ###

We want the user to give importance to more than just the description, we created a cosine similarity for the topics but also weighted it for the episodes and the anime popularity, this leads to more than just giving back the animes that talk about 

In [110]:
for elem in range(0,100):
    episodes_importance = int(input("How important is the episode number from 1 to 10?: "))
    desired_episodes = int(input("How many episodes would you like to watch?: "))
    if episodes_importance <= 10 and episodes_importance >0:
        break

How important is the episode number from 1 to 10?: 10
How many episodes would you like to watch?: 35


In [111]:
for elem in range(0,100):
    pop_imp = int(input("How important is the popularity from 1 to 10? "))
    if pop_imp <= 10 and pop_imp >0:
        break
popularity_ranks = []
for elem in range(0,19067,1906):
    popularity_ranks.append(elem)

How important is the popularity from 1 to 10? 10
What rank of popularity from 1 to 10?: 3


In [119]:
for elem in range(0,10000):
    topics = str(input("What would you like the anime to talk about? 3 word max "))
    query = topics.split(" ")
    if len(query) >1 and  len(query) <4:
        break

What would you like the anime to talk about? 3 word max kill man


In [120]:
index = search_engine_2(topics,df)

Function to create the final table to upload the results into

In [130]:
def display_results(query,cosine,episodes,popularity):
    disp_res = pd.DataFrame()
    disp_res = pd.DataFrame(columns = ['Anime Title','Anime URL','cosine_core','episode_score','popularity_score','final_score'])
    res = []
    for i in range(0,len(query)):
        roba = []
        roba.append(df.iloc[query[i][0]]['animeTitle'])
        roba.append(ReturnUrl(query[i][0]))
        roba.append(cosine[i])
        roba.append(final_score_episodes[i])
        roba.append(final_popularity_score[i])
        roba.append((cosine[i]+final_score_episodes[i]*episodes_importance+final_popularity_score[i]*pop_imp)/(1+1*episodes_importance+1*pop_imp))
        res.append(roba)
    index = 0
    for elem in res:
        disp_res.loc[index] = elem
        index += 1
    return(disp_res)

Function to calculate the cosine similarity for the TF_IDFS§

In [131]:
query_idf = compute_query_tfidf(compute_query_tf(topics))
final_query = []
for elem in query_idf:
    final_query.append(elem[1])
import numpy as np
def cosine_similarity(query,matches):
    matches_scores = {}
    for elem in matches:
        doc = matches[elem][0]
        matches_scores[elem] = round(np.dot(query,doc)/(np.linalg.norm(query)*np.linalg.norm(doc)),2)
    return(sorted(matches_scores.items(), key=lambda matches_scores: matches_scores[1],reverse = True))
final_result = (cosine_similarity(final_query,index))
cosine_scores = []
for elem in final_result:
    cosine_scores.append(elem[1])

Episode Score Contribution, for this part i defined 2 lists of 20 elements to calculate on a scale from 1 to 10 how our actual episodes score in reference to the user likings

In [132]:
final_score_episodes = []
for elem in final_result:
    actual_episodes = int(df.iloc[elem[0]]["animeNumEpisode"])
    episode_lower_distribution = []
    for elem in range(0,desired_episodes, desired_episodes//10):
        episode_lower_distribution.append(elem)
    episode_upper_distribution = []
    for elem in range(desired_episodes, desired_episodes*2, desired_episodes//10):
        episode_upper_distribution.append(elem)
    cont = 10
    if actual_episodes >= episode_upper_distribution[-1]:
        final_score_episodes.append(0.1)
    else:
        if actual_episodes == episode_lower_distribution[-1]:
            final_score_episodes.append(0.9)
        for score in range(1,10):
            if actual_episodes > episode_lower_distribution[-1]:
                cont -= 1
                if actual_episodes >= episode_upper_distribution[score-1] and actual_episodes< episode_upper_distribution[score] :
                    final_score_episodes.append((cont/10))
            else:
                if actual_episodes >= episode_lower_distribution[score-1] and actual_episodes < episode_lower_distribution[score] :
                    final_score_episodes.append((score/10))

For this part instead i defined a list of 10 popularity ranks to see how our anime popularity scores in respect to the user likings

In [135]:
final_popularity_score = []
for elem in final_result:
    anime = elem[0]
    actual_pop = int(df.iloc[anime]["animePopularity"])
    score = 0
    for i in range(1,len(popularity_ranks)):
        if actual_pop >= popularity_ranks[i-1] and actual_pop < popularity_ranks[i]:
            final_popularity_score.append(i/10)

Here i simply fetch the results and put them in a table

In [136]:
result = display_results(final_result,cosine_scores,final_score_episodes,final_popularity_score)
result.sort_values(by=["final_score"],ascending = False)

,Anime Title,Anime URL,cosine_core,episode_score,popularity_score,final_score
10,Ninpuu Kamui Gaiden,https://myanimelist.net/anime/5023/Ninpuu_Kamu...,1.00,0.9,0.6,0.761905
6,Ayashi no Ceres,https://myanimelist.net/anime/104/Ayashi_no_Ce...,1.00,0.9,0.2,0.571429
4,Mugen no Juunin: Immortal,https://myanimelist.net/anime/39806/Mugen_no_J...,1.00,0.9,0.2,0.571429
21,Gun x Sword,https://myanimelist.net/anime/411/Gun_x_Sword\n,0.80,0.9,0.2,0.561905
1,Death Note,https://myanimelist.net/anime/1535/Death_Note\n,1.00,0.9,0.1,0.523810
0,Vinland Saga,https://myanimelist.net/anime/37521/Vinland_Sa...,1.00,0.9,0.1,0.523810
14,Koroshiya-san: The Hired Gun,https://myanimelist.net/anime/20329/Koroshiya-...,1.00,0.4,0.4,0.428571
15,Legend of Duo,https://myanimelist.net/anime/600/Legend_of_Duo\n,1.00,0.5,0.3,0.428571
16,Shin Karate Jigoku-hen,https://myanimelist.net/anime/20963/Shimajirou...,1.00,0.1,0.7,0.428571
9,Blade of the Immortal,https://myanimelist.net/anime/4151/Blade_of_th...,1.00,0.5,0.2,0.380952


## PART 4 ##

In [2]:
#look for particular tags to get the reviews
def saveReviews(rev,soup):
    revs=soup.find_all("div",{"class":"spaceit textReadability word-break pt8 mt8",
                     "style":"clear: both; border-top: 1px solid #ebebeb;"})
    if len(revs)>1:
        for i in range(len(revs)):
            single_rev=revs[i].getText().strip().replace("\n","")[80:580]
            rev.write(single_rev+"\n")

In [ ]:
#Save the reviews
for page in tqdm(range(0,382)):

    start=50*page

    for i in range(start,start+50):
        #Open the articles of the page and save all the revies in a single directory
        os.chdir("Page"+str(page+1))
        art=open("article"+str(i+1)+'.html',"r")
        os.chdir("..")
        os.chdir("Reviews")
        rev=open("review_"+str(i+1)+".txt","w")

        soup=BeautifulSoup(art.read(), "html.parser") 

        #Just use the saving function we developed earlier
        saveReviews(rev,soup)

        art.close()
        rev.close()
        os.chdir("..")

In [ ]:
#Function that applies tokenize,lower,removes stopwords and stemming to a given string
def clean(string):
    tokenizer = RegexpTokenizer(r'\w+')
    string=tokenizer.tokenize(string)
    for word in range(len(string)):
        string[word] = string[word].lower() 
    #Remove stopwords
    string = [word for word in string if not word in stopwords.words()]
    #STEMMING
    stemmer = PorterStemmer()
    string = [stemmer.stem(word) for word in string]
    return str(" ".join(string))

In [ ]:
art_comment=[]
os.chdir("Reviews")
comments=[]
art_comment=[]
for page in tqdm(range(0,19083)):
    if os.stat("review_"+str(page+1)+".txt").st_size != 0: #do all the stuff below if the file is bigger than 0 bytes,
                                                           #so just do it if it's not empty
        rev=open("review_"+str(page+1)+".txt","r")
        lines=rev.readlines()
        for i in range(len(lines)):
            art_comment.append(clean(lines[i]))
        comments.append(str(art_comment)+"\n")            #we are saving as a list of lists the "cleaned" comments for 
                                                          #every page like so
                                                          #[[comment1page1,comment2page1,..][comment1page2,..]...]
        art_comment=[]
        rev.close()
    else:
        comments.append("\t\n")
    
os.chdir("..")

In [ ]:
#save results
os.chdir("Reviews")
c=open("comments.txt","w")
for comment in comments:
    c.write(str(comment))
c.close()
os.chdir("..")

In [3]:
#load results
comments=[]
c=open("comments.txt","r")
lines=c.readlines()
for i in range(len(lines)):
    comments.append(lines[i].replace("\n","").replace("[","").replace("]","").replace("'",'').split(","))
c.close()

In [7]:
#build corpus with the single words appearing in all reviews
corpus={}
#update=1
for i in tqdm(range(len(comments))):
    #print("i={}".format(i))
    for j in range(len(comments[i])):
        sentence=comments[i][j].split(" ")
        
        for word in sentence:
            
            if word not in corpus:
                corpus.update({word:0})
                #update=update+1
                

100%|██████████| 19083/19083 [00:00<00:00, 86610.34it/s]


In [8]:
#build vector with the count of words appearing in all reviews
vectors=[]
vec_sentence=[]
doc=dict.fromkeys(corpus, 0)
for i in tqdm(range(0,len(comments))):
    vec_sentence=[]
    for j in range(len(comments[i])):
        
        sentence=comments[i][j].split(" ")
        for word in sentence:
            doc[word] += 1
        
        vec_sentence.append(list(doc.values()))
        doc=dict.fromkeys(corpus, 0)
    vectors.append(vec_sentence)

100%|██████████| 19083/19083 [01:19<00:00, 239.03it/s]


In [9]:
#dimensions are fine
print(len(corpus))
print(len(comments[:][:]))
len(vectors)

36595
19083


19083

In [12]:
X=[]
for i in range(len(vectors)):
    for j in range(len(vectors[i])):
        X.append(vectors[i][j])
X=np.array(X)

In [14]:
#free some space on memory
def release_list(a):
    del a[:]
    del a
release_list(vectors)

In [15]:
kmeans = KMeans(init="random", n_clusters=2, n_init=30, max_iter=5000).fit(X)
y=(kmeans.labels_)
print(y)

[1 1 1 ... 1 1 0]


In [16]:
num_zeros = (y == 0).sum()
num_ones = (y == 1).sum()
print(num_zeros,num_ones)

13192 23493


We are applying KMeans (k=2) to vectors belonging to $R^{36595}$. The results of the clustering clearly show a realistic result, but it's hard to quantify how accurate the result is.\
Probably a better way to solve the problem would be implementing a neural network, but it would take a lot of time to assign the labels to the comments (those are needed for the training). Or maybe it could also work to train the neural network with the results obtained from the clustering, but we can't estimate how accurate the KMeans is and the error related to this would carry over the neural network itself.

## PART 5 ##

### 5.1 ####

In [ ]:
#Pseudocode:
"""
A: array of length l of which we want to compute the maximum sum of non adjacent elements

def compute(A): 
    l=len(A)       
    gain=[]      declare 2 arrays: one to store the values of the gain at each iteration and the other 
    path=[]                        to use later to save only the elements of the gain we care about (the path chosen
                                   to get the sum)
    inc=exc=0    here we initialize the variables that will keep inside them the value of the 2 sums at each iteration:
                 one including and one excluding the single element 
                 
    for i in range(l):          then we just compute the value of the 2 sums at each iteration and save the gain 
        temp=inc                for each of them
        inc=max(inc,A[i]+exc)
        exc=temp            
        gain[i]=(inc-exc)
    
    for j in range(0,len(gain)-1,2):  
        if gain[j+1]>0:               here we compare the elements of the gain list and take only those of interest:
            path[j]=(A[j+1])          if the gain is equal to 0 for that iteration the value is excluded "a priori"
        else:                         then we impose the condition to only get alternate values for which the gain is
            path[j]=(A[j])            greater than zero
    
    return sum(path),path
"""

### 5.2 ####

In [106]:
#Implementation
def compute(A):
    l=len(A)
    gain=[]
    path=[]
    inc=exc=0 #include, exclude --> compute value of sum including or excluding the i-th element

    for i in range(l):
        temp=inc
        inc=max(inc,A[i]+exc)
        exc=temp            
        gain.append(inc-exc)#save the gain

    
    for j in range(0,len(gain)-1,2):
        if gain[j+1]>0:
            path.append(A[j+1])
        else:
            path.append(A[j])
    
    return sum(path),path

Given an array A of length $n$ the complexity of the algorithm is $O(n)$ [excluding constants].\
It took me over 8 hours to come up with this solution (I had problems with the implementation), in which I tried different approaches to the problem:

1. I first tried looking at the list as a tree, but computing every possible case to then take the one with the highest output would lead me to a solution of order $O(n^2)$.
1. After this attempt I started searching on the internet and came across this video (https://www.youtube.com/watch?v=UtGtF6nc35g) in which the inclusion/exclusion method is explained. Making few additions to that algorithm I managed to get a result that is able to find the highest value solution along with the chosen elements of the starting array that seems to work in all the cases and in $O(n)$.